# Market Response Models

In [2]:
from __future__ import division
from datetime import datetime, timedelta,date

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import chart_studio.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go
pyoff.init_notebook_mode()

import sklearn
from sklearn.cluster import KMeans
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import classification_report,confusion_matrix

In [3]:
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

In [4]:
df_data = pd.read_csv('../datasets/market_response_model_data.csv')

In [5]:
df_data.head(10)

recency  history  used_discount  used_bogo   zip_code  is_referral channel  \
0       10   142.44              1          0  Surburban            0   Phone   
1        6   329.08              1          1      Rural            1     Web   
2        7   180.65              0          1  Surburban            1     Web   
3        9   675.83              1          0      Rural            1     Web   
4        2    45.34              1          0      Urban            0     Web   
5        6   134.83              0          1  Surburban            0   Phone   
6        9   280.20              1          0  Surburban            1   Phone   
7        9    46.42              0          1      Urban            0   Phone   
8        9   675.07              1          1      Rural            1   Phone   
9       10    32.84              0          1      Urban            1     Web   

             offer  conversion  
0  Buy One Get One           0  
1         No Offer           0  
2  Buy One Get One           0  
3         Discount           0  
4  Buy One Get One           0  
5  Buy One Get One           1  
6  Buy One Get One           0  
7  Buy One Get One           0  
8         Discount           0  
9  Buy One Get One           0

In [6]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64000 entries, 0 to 63999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   recency        64000 non-null  int64  
 1   history        64000 non-null  float64
 2   used_discount  64000 non-null  int64  
 3   used_bogo      64000 non-null  int64  
 4   zip_code       64000 non-null  object 
 5   is_referral    64000 non-null  int64  
 6   channel        64000 non-null  object 
 7   offer          64000 non-null  object 
 8   conversion     64000 non-null  int64  
dtypes: float64(1), int64(5), object(3)
memory usage: 4.4+ MB


In [7]:
df_data.conversion.mean()

0.14678125

In [8]:
def calc_uplift(df):
    #assigning 25$ to the average order value
    avg_order_value = 25
    
    #calculate conversions for each offer type
    base_conv = df[df.offer == 'No Offer']['conversion'].mean()
    disc_conv = df[df.offer == 'Discount']['conversion'].mean()
    bogo_conv = df[df.offer == 'Buy One Get One']['conversion'].mean()
    
    #calculate conversion uplift for discount and bogo
    disc_conv_uplift = disc_conv - base_conv
    bogo_conv_uplift = bogo_conv - base_conv
    
    #calculate order uplift
    disc_order_uplift = disc_conv_uplift * len(df[df.offer == 'Discount']['conversion'])
    bogo_order_uplift = bogo_conv_uplift * len(df[df.offer == 'Buy One Get One']['conversion'])
    
    #calculate revenue uplift
    disc_rev_uplift = disc_order_uplift * avg_order_value
    bogo_rev_uplift = bogo_order_uplift * avg_order_value
    
    
    print('Discount Conversion Uplift: {0}%'.format(np.round(disc_conv_uplift*100,2)))
    print('Discount Order Uplift: {0}'.format(np.round(disc_order_uplift,2)))
    print('Discount Revenue Uplift: ${0}\n'.format(np.round(disc_rev_uplift,2)))
          
    print('-------------- \n')

    print('BOGO Conversion Uplift: {0}%'.format(np.round(bogo_conv_uplift*100,2)))
    print('BOGO Order Uplift: {0}'.format(np.round(bogo_order_uplift,2)))
    print('BOGO Revenue Uplift: ${0}'.format(np.round(bogo_rev_uplift,2)))     

In [9]:
calc_uplift(df_data)

Discount Conversion Uplift: 7.66%
Discount Order Uplift: 1631.89
Discount Revenue Uplift: $40797.35

-------------- 

BOGO Conversion Uplift: 4.52%
BOGO Order Uplift: 967.4
BOGO Revenue Uplift: $24185.01


In [10]:
df_plot = df_data.groupby('recency').conversion.mean().reset_index()
plot_data = [
    go.Bar(
        x=df_plot['recency'],
        y=df_plot['conversion'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Recency vs Conversion',
        plot_bgcolor  = 'rgb(243,243,243)',
        paper_bgcolor  = 'rgb(243,243,243)',
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [11]:
kmeans = KMeans(n_clusters=5)
kmeans.fit(df_data[['history']])
df_data['history_cluster'] = kmeans.predict(df_data[['history']])

In [12]:
df_data = order_cluster('history_cluster', 'history',df_data,True)

In [13]:
df_data.groupby('history_cluster').agg({'history':['mean','min','max'], 'conversion':['count', 'mean']})

history                   conversion          
                        mean      min      max      count      mean
history_cluster                                                    
0                  73.504288    29.99   159.06      32127  0.122576
1                 245.028726   159.08   360.51      18000  0.159389
2                 476.253134   360.54   642.96       9184  0.180858
3                 809.311205   643.25  1110.09       3769  0.192624
4                1410.097750  1111.09  3345.93        920  0.217391

In [14]:
df_plot = df_data.groupby('history_cluster').conversion.mean().reset_index()
plot_data = [
    go.Bar(
        x=df_plot['history_cluster'],
        y=df_plot['conversion'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='History vs Conversion',
        plot_bgcolor  = 'rgb(243,243,243)',
        paper_bgcolor  = 'rgb(243,243,243)',
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [15]:
df_data.groupby(['used_discount','offer']).agg({'conversion':'mean'})

conversion
used_discount offer                      
0             Buy One Get One    0.169794
              Discount           0.166388
              No Offer           0.095808
1             Buy One Get One    0.136286
              Discount           0.196098
              No Offer           0.114533

In [16]:
df_data.groupby(['used_bogo','offer']).agg({'conversion':'mean'})

conversion
used_bogo offer                      
0         Buy One Get One    0.110892
          Discount           0.168968
          No Offer           0.099813
1         Buy One Get One    0.184530
          Discount           0.193974
          No Offer           0.111416

In [17]:
df_data.groupby(['used_discount','used_bogo','offer']).agg({'conversion':'mean'})

conversion
used_discount used_bogo offer                      
0             1         Buy One Get One    0.169794
                        Discount           0.166388
                        No Offer           0.095808
1             0         Buy One Get One    0.110892
                        Discount           0.168968
                        No Offer           0.099813
              1         Buy One Get One    0.251653
                        Discount           0.314993
                        No Offer           0.180549

In [18]:
df_plot = df_data.groupby('zip_code').conversion.mean().reset_index()
plot_data = [
    go.Bar(
        x=df_plot['zip_code'],
        y=df_plot['conversion'],
        marker=dict(
        color=['green', 'blue', 'orange'])
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Zip Code vs Conversion',
        plot_bgcolor  = 'rgb(243,243,243)',
        paper_bgcolor  = 'rgb(243,243,243)',
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [19]:
df_plot = df_data.groupby('is_referral').conversion.mean().reset_index()
plot_data = [
    go.Bar(
        x=df_plot['is_referral'],
        y=df_plot['conversion'],
        marker=dict(
        color=['green', 'blue', 'orange'])
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Referrals Conversion',
        plot_bgcolor  = 'rgb(243,243,243)',
        paper_bgcolor  = 'rgb(243,243,243)',
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [20]:
df_plot = df_data.groupby('channel').conversion.mean().reset_index()
plot_data = [
    go.Bar(
        x=df_plot['channel'],
        y=df_plot['conversion'],
        marker=dict(
        color=['green', 'blue', 'orange'])
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Channel vs Conversion',
        plot_bgcolor  = 'rgb(243,243,243)',
        paper_bgcolor  = 'rgb(243,243,243)',
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [21]:
df_plot = df_data.groupby('offer').conversion.mean().reset_index()
plot_data = [
    go.Bar(
        x=df_plot['offer'],
        y=df_plot['conversion'],
        marker=dict(
        color=['green', 'blue', 'orange'])
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Offer vs Conversion',
        plot_bgcolor  = 'rgb(243,243,243)',
        paper_bgcolor  = 'rgb(243,243,243)',
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [22]:
df_model = df_data.copy()
df_model = pd.get_dummies(df_model)

In [23]:
df_model.head()

recency  history  used_discount  used_bogo  is_referral  conversion  \
0       10   142.44              1          0            0           0   
1        2    45.34              1          0            0           0   
2        6   134.83              0          1            0           1   
3        9    46.42              0          1            0           0   
4       10    32.84              0          1            1           0   

   history_cluster  zip_code_Rural  zip_code_Surburban  zip_code_Urban  \
0                0               0                   1               0   
1                0               0                   0               1   
2                0               0                   1               0   
3                0               0                   0               1   
4                0               0                   0               1   

   channel_Multichannel  channel_Phone  channel_Web  offer_Buy One Get One  \
0                     0              1            0                      1   
1                     0              0            1                      1   
2                     0              1            0                      1   
3                     0              1            0                      1   
4                     0              0            1                      1   

   offer_Discount  offer_No Offer  
0               0               0  
1               0               0  
2               0               0  
3               0               0  
4               0               0

In [24]:
df_model.conversion.mean()

0.14678125

In [25]:
#create feature set and labels
X = df_model.drop(['conversion'],axis=1)
y = df_model.conversion

In [26]:
X.columns

Index(['recency', 'history', 'used_discount', 'used_bogo', 'is_referral',
       'history_cluster', 'zip_code_Rural', 'zip_code_Surburban',
       'zip_code_Urban', 'channel_Multichannel', 'channel_Phone',
       'channel_Web', 'offer_Buy One Get One', 'offer_Discount',
       'offer_No Offer'],
      dtype='object')

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=56)

In [28]:
xgb_model = xgb.XGBClassifier().fit(X_train, y_train)

In [29]:
X_test['proba'] = xgb_model.predict_proba(X_test)[:,1] 

In [30]:
X_test.head(5)

recency  history  used_discount  used_bogo  is_referral  \
32277        6   179.51              0          1            0   
12824       11   128.43              0          1            1   
20159        5    66.70              0          1            0   
41575        4   257.51              0          1            1   
10736        9    29.99              1          0            1   

       history_cluster  zip_code_Rural  zip_code_Surburban  zip_code_Urban  \
32277                1               0                   0               1   
12824                0               0                   1               0   
20159                0               0                   0               1   
41575                1               0                   0               1   
10736                0               0                   0               1   

       channel_Multichannel  channel_Phone  channel_Web  \
32277                     0              1            0   
12824                     0              0            1   
20159                     0              1            0   
41575                     0              0            1   
10736                     0              0            1   

       offer_Buy One Get One  offer_Discount  offer_No Offer     proba  
32277                      1               0               0  0.165668  
12824                      0               0               1  0.046888  
20159                      1               0               0  0.136825  
41575                      1               0               0  0.128528  
10736                      1               0               0  0.056537

In [31]:
X_test.proba.mean()

0.14546452

In [32]:
y_test.mean()

0.144453125

In [33]:
X_test['conversion'] = y_test

In [34]:
X_test[X_test['offer_Buy One Get One'] == 1].conversion.mean()

0.1502903600464576

In [35]:
X_test[X_test['offer_Buy One Get One'] == 1].proba.mean()

0.15082426

In [36]:
X_test[X_test['offer_Discount'] == 1].conversion.mean()

0.1784351145038168

In [37]:
X_test[X_test['offer_Discount'] == 1].proba.mean()

0.18159771

In [38]:
X_test[X_test['offer_No Offer'] == 1].conversion.mean()

0.10550778526609342

In [39]:
X_test[X_test['offer_No Offer'] == 1].proba.mean()

0.10490121

In [40]:
real_disc_uptick = int(len(X_test)*(X_test[X_test['offer_Discount'] == 1].conversion.mean() - X_test[X_test['offer_No Offer'] == 1].conversion.mean()))

pred_disc_uptick = int(len(X_test)*(X_test[X_test['offer_Discount'] == 1].proba.mean() - X_test[X_test['offer_No Offer'] == 1].proba.mean()))



In [41]:
print('Real Discount Uptick - Order: {}, Revenue: {}'.format(real_disc_uptick, real_disc_uptick*25))
print('Predicted Discount Uptick - Order: {}, Revenue: {}'.format(pred_disc_uptick, pred_disc_uptick*25))

Real Discount Uptick - Order: 933, Revenue: 23325
Predicted Discount Uptick - Order: 981, Revenue: 24525


In [42]:
real_bogo_uptick = int(len(X_test)*(X_test[X_test['offer_Buy One Get One'] == 1].conversion.mean() - X_test[X_test['offer_No Offer'] == 1].conversion.mean()))

pred_bogo_uptick = int(len(X_test)*(X_test[X_test['offer_Buy One Get One'] == 1].proba.mean() - X_test[X_test['offer_No Offer'] == 1].proba.mean()))



In [43]:
print('Real Discount Uptick - Order: {}, Revenue: {}'.format(real_bogo_uptick, real_bogo_uptick*25))
print('Predicted Discount Uptick - Order: {}, Revenue: {}'.format(pred_bogo_uptick, pred_bogo_uptick*25))

Real Discount Uptick - Order: 573, Revenue: 14325
Predicted Discount Uptick - Order: 587, Revenue: 14675
